In [1]:
import numpy as np
import pandas as pd
import os 

In [2]:
aas_residualmz={"G":57.02147,"A":71.03712,"S":87.03203,"P":97.05277,"V":99.06842,"T":101.04768,"C":103.00919,"I":113.08407,"L":113.08407,"N":114.04293
,"D":115.02695,"Q":128.05858,"K":128.09497,"E":129.0426,"M":131.04049,"H":137.05891,"F":147.06842,"R":156.10112,"Y":163.06333,"W":186.07932}
aas=['A' ,'C', 'D' ,'E', 'F', 'G', 'H', 'I' ,'K', 'L', 'M', 'N' ,'P' ,'Q' ,'R' ,'S', 'T', 'V', 'W' ,'Y']

In [3]:
path="C:\\Workplace\\C#\\Test\\immonium_mass_neh\\immonium_mass\\bin\\x64\\Debug\\"

data=pd.DataFrame()
for f in ["Liver_l0.csv"]:    
    data=pd.concat([data,pd.read_csv(path+f)])
    
    
data["ion_count"]=[ len(str(x).split('|')) for x in data[["values"]].values]
print(data.shape)
data=data[data.ion_count>=2]
data=data.reset_index(drop=True)
print(data.shape)

(26187, 9)
(15339, 9)


In [4]:
all_mz=pd.DataFrame(np.unique([ int(float(j)*100)/100 for i in [x for x in [x.split("|") for x in data["values"]] ] for j in i], return_counts=True)).transpose()
all_mz.columns=["mz","freq"]
all_mz=all_mz[all_mz.freq>100]

In [5]:
dict_res={x:{"count":0,"total":0,"aas":[],"peps":[],"all_mzs":[]} for x in all_mz.mz}
count=0
for i in range(data.shape[0]): 
    temp=np.unique([ int(float(x)*100)/100 for x in data["values"][i].split("|")])
    temp=[x for x in temp if x in list(all_mz.mz)]
    if len(temp)==0:
        continue
        
    for mz in all_mz[all_mz.mz>= min(temp)].mz:
        dict_res[mz]["total"]+=1        
    
    for mz in temp:
        dict_res[mz]["count"]+=1
        dict_res[mz]["aas"]+=list(np.unique(list(data["peptide"][i])))
        dict_res[mz]["peps"]+=[data["peptide"][i]]
        dict_res[mz]["all_mzs"]+=list(np.unique(temp))
    

In [6]:
# np.unique(dict_res[list(dict_res.keys())[0]]["all_mzs"],return_counts=True)

In [7]:
# selected_res=[]
# for k in dict_res.keys():
# #     print(k,dict_res[k]["count"],dict_res[k]["total"],"\t===>\t",100*dict_res[k]["count"]/dict_res[k]["total"])
#     selected_res.append([k,dict_res[k]["count"],dict_res[k]["total"],100*dict_res[k]["count"]/dict_res[k]["total"]])

In [8]:
# selected_res=pd.DataFrame(selected_res)
# selected_res.columns=["mz","freq","Total","perc"]
# selected_res=selected_res.sort_values(by="perc",ascending=False).reset_index(drop=True)
# selected_res.to_csv("possible_mz.csv",index=False)
# selected_res.head(50)

In [18]:
selected_res=[]
for k in dict_res.keys():
    
    temp=(pd.DataFrame(np.unique(dict_res[k]["all_mzs"] ,return_counts=True)).transpose())
    temp.columns=["mz","perc"]
    temp.perc=100*temp.perc/dict_res[k]["count"]
    temp=temp.sort_values(by="perc",ascending=False)
    temp=(temp[temp.perc>30].reset_index(drop=True))
    if (temp.shape[0]>0):
        print("mz:",k,"count:",f'{dict_res[k]["count"]}({int(1000*dict_res[k]["count"]/dict_res[k]["total"])/10})',"Range available:",dict_res[k]["total"],"\n")
        print(temp[temp.mz!=k].head(12))
        print("------------------------------------------\n")
  

mz: 65.39 count: 185(100.0) Range available: 185 

        mz       perc
1    86.09  99.459459
2    70.06  98.378378
3    72.08  97.297297
4   129.10  95.675676
5   120.08  94.594595
6    84.08  93.513514
7   110.07  88.108108
8    84.04  76.216216
9    87.10  69.189189
10   74.06  62.162162
11  112.65  61.081081
12  101.07  60.540541
------------------------------------------

mz: 69.07 count: 383(76.1) Range available: 503 

        mz        perc
0    86.09  100.000000
2    70.06   97.389034
3    72.08   96.344648
4   129.10   96.083551
5    84.08   94.778068
6   120.08   91.644909
7    87.10   91.644909
8   110.07   86.422977
9    84.04   78.590078
10  101.07   66.057441
11   74.06   63.185379
12  102.05   61.879896
------------------------------------------

mz: 70.02 count: 135(23.3) Range available: 577 

        mz        perc
1    86.09  100.000000
2   129.10   97.777778
3    70.06   97.777778
4    72.08   97.777778
5    84.08   97.777778
6   120.08   95.555556
7   110.07   89

mz: 89.02 count: 450(8.1) Range available: 5548 

        mz       perc
1   129.10  94.888889
2   110.07  87.333333
3    86.09  86.000000
4    99.80  74.222222
5   120.08  73.777778
6    84.08  71.777778
7   101.07  66.444444
8    84.04  59.555556
9   102.05  56.000000
10  112.66  55.111111
11   87.10  40.000000
12  104.05  38.888889
------------------------------------------

mz: 89.07 count: 204(3.6) Range available: 5553 

        mz       perc
1   129.10  96.568627
2   110.07  94.607843
3   101.07  91.666667
4   120.08  91.176471
5    86.09  87.254902
6    84.08  70.588235
7   102.05  67.647059
8    84.04  67.647059
9   115.08  50.490196
10   87.10  50.490196
11  129.06  50.000000
12  104.05  48.529412
------------------------------------------

mz: 91.05 count: 1242(21.6) Range available: 5747 

        mz       perc
1   129.10  97.181965
2   110.07  93.961353
3   120.08  92.512077
4   101.07  78.663446
5    86.09  71.578100
6   102.05  69.404187
7   104.05  62.077295
8    84.08  

mz: 112.65 count: 5257(44.8) Range available: 11713 

        mz       perc
1   129.10  92.238920
2   120.08  86.151798
3   110.07  80.483165
4   101.07  51.949781
5    99.79  43.998478
6   102.05  43.884345
7    86.09  39.699448
8   128.17  39.661404
9   104.05  37.607000
10  129.06  35.096062
11   84.08  32.946547
12  115.08  31.272589
------------------------------------------

mz: 112.66 count: 3298(27.5) Range available: 11968 

        mz       perc
1   129.10  91.752577
2   110.07  80.260764
3   120.08  79.290479
4   101.07  50.879321
5   128.18  44.147968
6    99.79  43.571862
7   102.05  42.086113
8    86.09  40.600364
9    84.08  33.808369
10  104.05  33.747726
11  129.06  31.412978
12  115.08  30.078836
------------------------------------------

mz: 112.67 count: 352(2.9) Range available: 11999 

        mz       perc
1   129.10  89.772727
2   110.07  78.977273
3   120.08  72.443182
4    99.80  54.261364
5   101.07  50.852273
6   128.19  46.590909
7    86.09  41.477273
8   

mz: 129.09 count: 485(3.1) Range available: 15180 

        mz       perc
1   129.10  99.587629
2   120.08  76.494845
3   110.07  74.226804
4   101.07  49.072165
5   102.05  47.422680
6   129.06  41.030928
7   101.10  39.587629
8   104.05  36.701031
9    86.09  36.082474
10  112.65  35.670103
11  115.08  35.463918
12   99.79  32.577320
------------------------------------------

mz: 129.1 count: 14047(91.6) Range available: 15322 

       mz       perc
1  120.08  77.988183
2  110.07  66.839895
3  101.07  43.382929
4  102.05  36.142949
5  129.06  35.915142
6  112.65  34.519826
7   86.09  32.718730
8  104.05  30.504734
------------------------------------------

mz: 129.11 count: 109(0.7) Range available: 15322 

        mz       perc
1   129.10  95.412844
2   120.08  76.146789
3   110.07  70.642202
4   112.08  57.798165
5   115.08  54.128440
6   129.06  48.623853
7   101.07  48.623853
8   116.07  45.871560
9   102.05  38.532110
10  112.65  35.779817
11   86.09  33.027523
---------------

In [17]:
selected_res=[]
for k in dict_res.keys():
    
    temp=(pd.DataFrame(np.unique(dict_res[k]["aas"] ,return_counts=True)).transpose())
    temp.columns=["aa","perc"]
    temp.perc=100*temp.perc/dict_res[k]["count"]
    temp=temp.sort_values(by="perc",ascending=False)
    temp=(temp[temp.perc>30].reset_index(drop=True))
    if (temp.shape[0]>0):
        print("mz:",k,"count:",f'{dict_res[k]["count"]}({int(1000*dict_res[k]["count"]/dict_res[k]["total"])/10})',"Range available:",dict_res[k]["total"],"\n")
        print(temp.head(9))
        print("------------------------------------------\n")
  

mz: 65.39 count: 185(100.0) Range available: 185 

  aa       perc
0  L   69.72973
1  A  58.918919
2  V  51.351351
3  K  50.810811
4  G  46.486486
5  R  45.405405
6  F  42.702703
7  I  36.216216
8  P  31.351351
------------------------------------------

mz: 69.07 count: 383(76.1) Range available: 503 

  aa       perc
0  I  71.801567
1  L  65.535248
2  K   58.48564
3  A  50.913838
4  V  48.563969
5  G  46.997389
6  R  38.120104
7  F  35.770235
8  S  32.114883
------------------------------------------

mz: 70.02 count: 135(23.3) Range available: 577 

  aa       perc
0  L  68.148148
1  K  54.074074
2  A  48.888889
3  V  45.185185
4  S  45.185185
5  R  44.444444
6  N  41.481481
7  F       40.0
8  G       40.0
------------------------------------------

mz: 70.06 count: 1265(98.9) Range available: 1279 

  aa       perc
0  L  66.798419
1  A  53.043478
2  V  52.490119
3  K  50.355731
4  R  48.458498
5  G    45.6917
6  I  42.766798
7  F  35.968379
8  S  34.071146
-------------------------

mz: 91.05 count: 1242(21.6) Range available: 5747 

  aa       perc
0  L  64.814815
1  K  54.347826
2  Y  53.140097
3  G  52.093398
4  A  49.838969
5  V  48.309179
6  D  45.652174
7  R  45.652174
8  E  45.491143
------------------------------------------

mz: 93.04 count: 463(7.9) Range available: 5851 

  aa       perc
0  H  79.049676
1  L  63.066955
2  R  53.347732
3  G  52.267819
4  V  50.539957
5  A  50.323974
6  D  48.164147
7  E  47.516199
8  K  46.436285
------------------------------------------

mz: 94.17 count: 103(1.7) Range available: 5874 

  aa       perc
0  L  63.106796
1  G  58.252427
2  V  54.368932
3  A  53.398058
4  R  51.456311
5  S  50.485437
6  K  47.572816
7  E  46.601942
8  F  42.718447
------------------------------------------

mz: 98.02 count: 176(2.9) Range available: 5917 

  aa       perc
0  N      68.75
1  L       62.5
2  K  60.795455
3  D  57.954545
4  V  54.545455
5  I  48.295455
6  G  45.454545
7  E  44.886364
8  A  44.318182
--------------------------

mz: 128.17 count: 3566(24.3) Range available: 14657 

  aa       perc
0  L  74.172743
1  A  59.197981
2  V   59.08581
3  G  58.917555
4  E  57.571509
5  S  54.234436
6  K  53.673584
7  D  51.037577
8  I  51.009534
------------------------------------------

mz: 128.18 count: 3357(22.4) Range available: 14925 

  aa       perc
0  L  71.075365
1  V  60.857909
2  A   60.82812
3  E  58.623771
4  G  57.700328
5  K  54.006553
6  S  52.755436
7  I  50.819184
8  D  50.551087
------------------------------------------

mz: 128.19 count: 652(4.3) Range available: 14978 

  aa       perc
0  L  68.404908
1  A  63.343558
2  G  59.969325
3  V  58.588957
4  S  54.754601
5  E  54.601227
6  K   53.98773
7  T  49.233129
8  D  48.312883
------------------------------------------

mz: 129.06 count: 5270(34.7) Range available: 15177 

  aa       perc
0  L  71.461101
1  A  62.087287
2  E  59.848197
3  V  59.127135
4  G  57.874763
5  Q  55.616698
6  S   53.73814
7  K  53.017078
8  I  50.588235
--------------

In [85]:
selected_res=[]
for k in dict_res.keys():
    
    possible_aa=list(set([key for key,val in aas_residualmz.items() if val >=k ])- set(['K','L','R']))
    
    temp=[]
    for i in range(len(possible_aa)):
        for j in range(i+1,len(possible_aa)):
            for pep in dict_res[k]["peps"]:
                if possible_aa[i] in pep and possible_aa[j] in pep:
                    temp.append("".join(sorted(possible_aa[i]+possible_aa[j])))               
    
    temp=(pd.DataFrame(np.unique(temp,return_counts=True)).transpose())
    temp.columns=["aa","perc"]
    temp.perc=100*temp.perc/dict_res[k]["count"]
    temp=temp.sort_values(by="perc",ascending=False)
    temp=(temp[temp.perc>50].reset_index(drop=True))
    if temp.shape[0]>0:
        print(k,dict_res[k]["count"],dict_res[k]["total"],"\t===>\t",100*dict_res[k]["count"]/dict_res[k]["total"],"\n")
        print(temp)
        print("------------------------------------------\n")
    
    
#     selected_res.append([k,dict_res[k]["count"],dict_res[k]["total"],100*dict_res[k]["count"]/dict_res[k]["total"]])

104.04 122 9187 	===>	 1.3279634265810385 

   aa       perc
0  EM  53.278689
------------------------------------------



In [16]:
[key for key,val in aas_residualmz.items() if val >=75 ]

['S',
 'P',
 'V',
 'T',
 'C',
 'I',
 'L',
 'N',
 'D',
 'Q',
 'K',
 'E',
 'M',
 'H',
 'F',
 'R',
 'Y',
 'W']